# IV Strategy Analysis

This notebook scores options opportunity across covered calls, cash-secured puts, and term-structure context.

**Modes**

- `QUICK_SCAN=1`: scorecard and top visuals only
- `QUICK_RUN=1`: reduced ticker set for faster execution


In [ ]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown

from notebook_pipeline import (
    setup_report_style,
    parse_env_list,
    display_table,
    screen_universe,
    fetch_underlying_metrics,
    fetch_option_candidates,
    score_option_candidates,
)

setup_report_style()

FIGURE_COUNTER = 0


def show_figure(fig, title):
    global FIGURE_COUNTER
    FIGURE_COUNTER += 1
    if not str(title).lower().startswith("figure"):
        title = f"Figure {FIGURE_COUNTER}. {title}"
    fig.update_layout(title=title)
    fig.show()


In [ ]:
USE_SCREEN = os.getenv("USE_SCREEN", "1") == "1"
QUICK_RUN = os.getenv("QUICK_RUN", "0") == "1"
QUICK_SCAN = os.getenv("QUICK_SCAN", "0") == "1"
TICKER_OVERRIDE = parse_env_list("TICKER_OVERRIDE")

MAX_TICKERS = int(os.getenv("MAX_TICKERS", "24"))
RATE_LIMIT_SLEEP = float(os.getenv("RATE_LIMIT_SLEEP", "0.20"))
TOP_N = int(os.getenv("TOP_N", "12"))


if QUICK_RUN:
    MAX_TICKERS = min(MAX_TICKERS, 10)
    RATE_LIMIT_SLEEP = min(RATE_LIMIT_SLEEP, 0.10)

if QUICK_SCAN:
    QUICK_RUN = True
    MAX_TICKERS = min(MAX_TICKERS, 12)

HORIZONS = {
    "short": {"min_dte": 21, "max_dte": 75, "target_dte": 45},
    "medium": {"min_dte": 76, "max_dte": 220, "target_dte": 140},
}
CALL_MONEYNESS = {"short": (0.95, 1.08), "medium": (0.90, 1.10)}
PUT_MONEYNESS = {"short": (0.92, 1.08), "medium": (0.88, 1.12)}

config_df = pd.DataFrame(
    {
        "Parameter": ["USE_SCREEN", "QUICK_RUN", "QUICK_SCAN", "MAX_TICKERS", "TOP_N"],
        "Value": [USE_SCREEN, QUICK_RUN, QUICK_SCAN, MAX_TICKERS, TOP_N],
    }
)
display_table(config_df, caption="IV Strategy Configuration")


In [ ]:
tickers = screen_universe(
    use_screen=USE_SCREEN,
    ticker_override=TICKER_OVERRIDE,
    max_tickers=MAX_TICKERS,
    size=max(60, MAX_TICKERS * 3),
)
metrics_df = fetch_underlying_metrics(
    tickers,
    history_period="1y",
    rate_limit_sleep=RATE_LIMIT_SLEEP,
)
if metrics_df.empty:
    raise RuntimeError("No underlyings available for IV strategy run.")

call_frames = []
put_frames = []
for _, row in metrics_df.iterrows():
    ticker = row["ticker"]
    spot = float(row["spot"])
    calls = fetch_option_candidates(
        ticker,
        side="call",
        spot=spot,
        horizons=HORIZONS,
        moneyness_bounds=CALL_MONEYNESS,
        rate_limit_sleep=RATE_LIMIT_SLEEP,
    )
    puts = fetch_option_candidates(
        ticker,
        side="put",
        spot=spot,
        horizons=HORIZONS,
        moneyness_bounds=PUT_MONEYNESS,
        rate_limit_sleep=RATE_LIMIT_SLEEP,
    )
    if not calls.empty:
        call_frames.append(calls)
    if not puts.empty:
        put_frames.append(puts)

call_df = pd.concat(call_frames, ignore_index=True) if call_frames else pd.DataFrame()
put_df = pd.concat(put_frames, ignore_index=True) if put_frames else pd.DataFrame()

scored_calls = (
    score_option_candidates(call_df, metrics_df)
    if not call_df.empty
    else pd.DataFrame()
)
scored_puts = (
    score_option_candidates(put_df, metrics_df) if not put_df.empty else pd.DataFrame()
)

if scored_calls.empty and scored_puts.empty:
    raise RuntimeError("No option candidates available for IV strategy analysis.")

short_calls = scored_calls[scored_calls["horizon"] == "short"].copy()
short_puts = scored_puts[scored_puts["horizon"] == "short"].copy()

cc_target = pd.DataFrame()
if not short_calls.empty:
    short_calls["atm_dist"] = (short_calls["moneyness"] - 1.02).abs()
    cc_target = (
        short_calls.sort_values(["ticker", "atm_dist", "dte"])
        .groupby("ticker", as_index=False)
        .first()
    )
    cc_target["cc_ann_yield"] = (cc_target["mid"] / cc_target["spot"]) * (
        365 / cc_target["dte"]
    )

csp_target = pd.DataFrame()
if not short_puts.empty:
    short_puts["atm_dist"] = (short_puts["moneyness"] - 0.98).abs()
    csp_target = (
        short_puts.sort_values(["ticker", "atm_dist", "dte"])
        .groupby("ticker", as_index=False)
        .first()
    )
    csp_target["csp_ann_yield"] = (csp_target["mid"] / csp_target["strike"]) * (
        365 / csp_target["dte"]
    )

term_df = pd.DataFrame()
if not scored_calls.empty:
    term_tmp = scored_calls.groupby(["ticker", "horizon"], as_index=False)[
        "iv"
    ].median()
    term_df = term_tmp.pivot(
        index="ticker", columns="horizon", values="iv"
    ).reset_index()
    if "short" in term_df.columns and "medium" in term_df.columns:
        term_df["term_slope"] = term_df["medium"] - term_df["short"]
    else:
        term_df["term_slope"] = np.nan

summary_df = metrics_df.copy()
if not cc_target.empty:
    summary_df = summary_df.merge(
        cc_target[["ticker", "cc_ann_yield", "iv"]].rename(
            columns={"iv": "atm_iv_call"}
        ),
        on="ticker",
        how="left",
    )
else:
    summary_df["cc_ann_yield"] = np.nan
    summary_df["atm_iv_call"] = np.nan

if not csp_target.empty:
    summary_df = summary_df.merge(
        csp_target[["ticker", "csp_ann_yield"]], on="ticker", how="left"
    )
else:
    summary_df["csp_ann_yield"] = np.nan

if not term_df.empty:
    summary_df = summary_df.merge(
        term_df[["ticker", "term_slope"]], on="ticker", how="left"
    )
else:
    summary_df["term_slope"] = np.nan

summary_df["fund_score"] = np.clip(
    50
    + summary_df["roe"].fillna(0.10) * 160
    + summary_df["rev_growth"].fillna(0.05) * 130
    + summary_df["profit_margin"].fillna(0.08) * 90,
    0,
    100,
)

summary_df["options_score"] = np.clip(
    40
    + summary_df["cc_ann_yield"].fillna(0) * 120
    + summary_df["csp_ann_yield"].fillna(0) * 90
    + summary_df["atm_iv_call"].fillna(0.35) * 50
    - summary_df["term_slope"].fillna(0) * 220,
    0,
    100,
)

summary_df["composite_score"] = (
    0.45 * summary_df["fund_score"] + 0.55 * summary_df["options_score"]
)
ranked = summary_df.sort_values("composite_score", ascending=False).head(TOP_N)

display_table(
    ranked[
        [
            "ticker",
            "sector",
            "spot",
            "fund_score",
            "options_score",
            "composite_score",
            "cc_ann_yield",
            "csp_ann_yield",
            "atm_iv_call",
            "term_slope",
        ]
    ],
    caption="IV Strategy Scorecard",
    format_dict={
        "spot": "${:,.2f}",
        "fund_score": "{:.1f}",
        "options_score": "{:.1f}",
        "composite_score": "{:.1f}",
        "cc_ann_yield": "{:.1%}",
        "csp_ann_yield": "{:.1%}",
        "atm_iv_call": "{:.1%}",
        "term_slope": "{:+.2%}",
    },
)


In [ ]:
fig_score = px.bar(
    ranked.sort_values("composite_score"),
    x="composite_score",
    y="ticker",
    orientation="h",
    color="options_score",
    color_continuous_scale="Blues",
    height=560,
)
show_figure(fig_score, "Composite Opportunity Score")

fig_scatter = px.scatter(
    ranked,
    x="fund_score",
    y="options_score",
    color="ticker",
    size="composite_score",
    hover_data=["cc_ann_yield", "csp_ann_yield", "term_slope"],
    height=520,
)
fig_scatter.update_xaxes(range=[0, 100])
fig_scatter.update_yaxes(range=[0, 100])
show_figure(fig_scatter, "Fundamental vs Options Opportunity")

if not term_df.empty:
    top_names = ranked["ticker"].tolist()
    term_long = (
        term_df[term_df["ticker"].isin(top_names)]
        .melt(
            id_vars=["ticker"],
            value_vars=[c for c in ["short", "medium"] if c in term_df.columns],
            var_name="horizon",
            value_name="iv",
        )
        .dropna()
    )
    if not term_long.empty:
        fig_term = px.line(
            term_long,
            x="horizon",
            y="iv",
            color="ticker",
            markers=True,
            height=500,
        )
        fig_term.update_yaxes(tickformat=".1%")
        show_figure(fig_term, "IV Term Structure (Top Ranked Names)")


In [ ]:
display(Markdown("## Run Summary"))

summary_lines = [
    f"- **Tickers analyzed:** {len(tickers)}",
    f"- **Total call contracts:** {len(call_df):,}",
    f"- **Total put contracts:** {len(put_df):,}",
    f"- **Ranked opportunities:** {len(ranked)}",
]
if not cc_target.empty:
    summary_lines.append(f"- **Covered-call targets:** {len(cc_target)}")
if not csp_target.empty:
    summary_lines.append(f"- **CSP targets:** {len(csp_target)}")
display(Markdown("\n".join(summary_lines)))

if not ranked.empty:
    display(Markdown("### Top Opportunities at a Glance"))
    display_table(
        ranked[["ticker", "composite_score", "fund_score", "options_score"]].head(10),
        caption="Top Ranked Opportunities",
        format_dict={
            "composite_score": "{:.1f}",
            "fund_score": "{:.1f}",
            "options_score": "{:.1f}",
        },
    )
